## Вытаскиваем формулы и URL-ы.

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %reset

In [ ]:
if __name__ == '__main__' and '__file__' not in globals():
    %run common_info.ipynb

In [ ]:
%run extract_images_tables_captions.ipynb

### Ищем все урлы, потому что потом из-за них неправильно бьются предложения.

In [ ]:
import re

def searchURLs(pageObjectsByY1List):
    urls = []
    urlPattern = re.compile(r'(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))')
    
    for pageObjectsByY1 in pageObjectsByY1List:
        for y1, pageObjects in pageObjectsByY1.items():
            for pageObject in pageObjects:         
                if isinstance(pageObject['object'], LTTextBox):
                    urls += urlPattern.findall(pageObject['object'].get_text())
        
    return urls

In [ ]:
urls = searchURLs(pageObjectsByY1List)

if __name__ == '__main__' and '__file__' not in globals():
    print(urls)

### Извлекаем из текста все необычные символы. Возможно, это пригодится при идентификации формул.

In [ ]:
from string import punctuation, whitespace
import re

usualSymbols = russianLetters.union(englishLetters)\
                             .union(russianUpperLetters)\
                             .union(englishUpperLetters)\
                             .union(numbers)\
                             .union(set(punctuation))\
                             .union(set(whitespace))\
                             .union(listMarkers)

def extractUnusualSymbols(pageObjectsByY1List):
    usedSymbols = set()
    
    for pageObjectsByY1 in pageObjectsByY1List:
        for y1, pageObjects in pageObjectsByY1.items():
            for pageObject in pageObjects:         
                if isinstance(pageObject['object'], LTTextBox):
                    # потому что это известные символы, которые неверно распознаются
                    objectText = pageObject['object'].get_text().replace('(cid:22)', '-').replace('(cid:88)\n', '').replace('(cid:190)', '«').replace('¿', '»')
                    cidSymbols = re.findall(r'\(cid:\d+\)', objectText)
                    usedSymbols = usedSymbols.union(set(re.sub(r'\(cid:\d+\)', '', objectText))).union(set(cidSymbols))
        
    return usedSymbols.difference(usualSymbols)

In [ ]:
unusualSymbols = extractUnusualSymbols(pageObjectsByY1List)

if __name__ == '__main__' and '__file__' not in globals():
    print(unusualSymbols)

### Пытаемся найти и отметить формулы. Алгоритм основан на эвристиках, некоторые формулы не отмечает. Отмечает ошибочно части в списке литературы (хотя список литературы не входит в презентацию, так что это приемлемо).

In [ ]:
from string import punctuation, whitespace
import re

# формула не содежит букв русского алфавита!
# пусть формула содержит минимум 19% пунктуации, 12% пробельных символов, 12% букв английского языка
# или больше 40% пунктуации (или необычных символов) и больше 40% букв английского языка
# или больше 24% пунктуации и необычных символов и больше 40% букв английского языка
# или больше 90% необычных символов
# или больше 50% пунктуации и необычных символов
def markFormulas(pageObjectsByY1List, unusualSymbols):
    formulas = []
    
    for pageObjectsByY1 in pageObjectsByY1List:
        for y1, pageObjects in pageObjectsByY1.items():
            for pageObject in pageObjects:         
                if isinstance(pageObject['object'], LTTextBox):
                    objectTextByLineFeed = pageObject['object'].get_text().split('\n')
                    objectTextByLineFeed = list(filter(None, objectTextByLineFeed))
                    
                    for objectTextPart in objectTextByLineFeed:
                        objectTextPartCopy = objectTextPart.replace('(cid:22)', '-').replace('(cid:88)\n', '').replace('(cid:190)', '«').replace('¿', '»')
#                         for us in unusualSymbols:
#                             objectTextPartCopy = objectTextPartCopy.replace(us, '')

                        if objectTextPartCopy != '':
                            objectTextPartLen = len(objectTextPartCopy)
                            russianLettersPercent = len(re.findall(r'[' + re.escape('абвгдеёжзийклмнопрстуфхцчшщъыьэюя') + re.escape('абвгдеёжзийклмнопрстуфхцчшщъыьэюя'.upper()) + r']', 
                                                                   objectTextPartCopy)) / objectTextPartLen * 100
                            unusualPercent = len(re.findall(r'' + unusualSymbols + r'', 
                                                            objectTextPartCopy)) / objectTextPartLen * 100
                            punctuationPercent = len(re.findall(r'[' + re.escape(punctuation) + r']', 
                                                                objectTextPartCopy)) / objectTextPartLen * 100
                            whitespacePercent = len(re.findall(r'[' + re.escape(whitespace) + r']', 
                                                               objectTextPartCopy)) / objectTextPartLen * 100
                            englishLettersPercent = len(re.findall(r'[' + re.escape('abcdefghijklmnopqrstuvwxyz') + re.escape('abcdefghijklmnopqrstuvwxyz'.upper()) + r']', 
                                                                   objectTextPartCopy)) / objectTextPartLen * 100
                            if (russianLettersPercent > 0.0):
                                continue
                            if (punctuationPercent >= 19.0 and whitespacePercent >= 12.0 and englishLettersPercent >= 12.0) or\
                               (punctuationPercent >= 40.0 and englishLettersPercent >= 40.0) or \
                               (unusualPercent >= 40.0 and englishLettersPercent >= 40.0) or \
                               (punctuationPercent+unusualPercent >= 24.0 and englishLettersPercent >= 25.0) or \
                               unusualPercent >= 90.0 or \
                               punctuationPercent+unusualPercent >= 50.0:
                                formulas.append(objectTextPart)
                                if len(objectTextByLineFeed) == 1:
                                    pageObject['label'] = 'formula'
                                else:
                                    pageObject['label'] = 'contains formula'
                                    if len(list(filter(lambda textPart: textPart == '' or textPart in formulas, objectTextByLineFeed))) == len(objectTextByLineFeed):
                                        pageObject['label'] = 'formula'
                                    
    return pageObjectsByY1List, formulas

In [ ]:
from pprint import pprint
import re

formulas = []
# pageObjectsByY1List, formulas = markFormulas(pageObjectsByY1List, '|'.join(re.escape(symbol) for symbol in unusualSymbols))

if __name__ == '__main__' and '__file__' not in globals():
    pprint(formulas)